In [ ]:
import os
import sys
import numpy as np
import math
import torch
from torch.utils.data import DataLoader
import create_paths
from create_paths import smoothen_paths
# from create_paths import mat_to_path
import matplotlib.pyplot as plt

import importlib.util
import sys
import data_manager as dm

Configure the evaluation run

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_grad_enabled(False)

DATASET = 'many_maps_1'      # (String) The name of the dataset to load. Leave as None to use the same dataset that the model was trained on.
SUBSET = 'eval'     # The name of the subset to load from

RUN_NAME = 'rare-waterfall-111'
STEP = 14

BATCH_SIZE = 50
SMOOTH_VAL = 3

NUM_SAMPLES = 5     # the number of example outputs to generate
FIG_SCALE = 20      # the number of pixels per inch in the generated plots

Import the gan's class definitions

In [ ]:
# Make sure the cell only runs once
try:
    if MODEL_IMPORTED:
        print("Skipping cell")
except:
    config = dm.load_gan(RUN_NAME)

    from GANdefs import Generator
    gen = Generator(config['features_gen'], config['kernels_gen'], config['stride_gen'], config['padding_gen'], device=DEVICE)

    if DATASET == None:
        DATASET = config['dataset']
        SUBSET = config['subset']

MODEL_IMPORTED = True

In [ ]:
def path_to_tensor(path, map_dim):
    path_mat = torch.zeros(map_dim)

    # Make the path continuous
    for i in range(path.shape[0] - 1):
        x = int(path[i,1])
        x1 = int(path[i,1])
        x2 = int(path[i+1,1])

        y = int(path[i,0])
        y1 = int(path[i,0])
        y2 = int(path[i+1,0])

        if (x1 < x2):
            x_dir = 1
        else:
            x_dir = -1

        if (y1 < y2):
            y_dir = 1
        else:
            y_dir = -1

        # Determine y from x
        if x2-x1 != 0:
            m = (y2-y1)/(x2-x1)
            while x != x2:
                y = round(m*(x-x1) + y1)
                path_mat[y,x] = 1
                x += x_dir
        else:
            while x != x2:
                path_mat[y1,x] = 1
                x += x_dir


        x = int(path[i,1])
        x1 = int(path[i,1])
        x2 = int(path[i+1,1])

        y = int(path[i,0])
        y1 = int(path[i,0])
        y2 = int(path[i+1,0])

        # Determine x from y
        if y2-y1 != 0:
            m = (x2-x1)/(y2-y1)
            while y != y2:
                x = round(m*(y-y1) + x1)
                path_mat[y,x] = 1
                y += y_dir
        else:
            while y != y2:
                path_mat[y,x1] = 1
                y += y_dir
        
    path_mat[int(path[path.shape[0]-1,0]), int(path[path.shape[0]-1,1])] = 1     # Include the last point in the path

    return torch.tensor(path_mat)

Load the network(s)

In [ ]:
# Make sure the cell only runs once
try:
    if MODEL_LOADED:
        print("Skipping cell execution")
except:

    state = dm.load_checkpoint(RUN_NAME, STEP)
    gen.load_state_dict(state['gen'])
    gen.eval()

MODEL_LOADED = True

Get the input set

In [ ]:
# Need to override __init__, __len__, __getitem__
# as per datasets requirement
class PathsDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, subset, device='cpu'):
        self.device = device
        self.paths = dm.load_input(dataset, subset) # Load all of the paths in the specified set

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        x = self.paths[idx]
        x = x.to(self.device)
        return x

In [ ]:
# Make sure the cell only runs once
try:
    if INPUT_LOADED:
        print("Skipping cell execution")
except:

    inputs = dm.load_input(DATASET, SUBSET)
    dataloader = DataLoader(inputs, batch_size = BATCH_SIZE, shuffle=True, drop_last = True)

INPUT_LOADED = True

Get the resulting outputs

In [ ]:
outputs = torch.Tensor()
for item_idx, item in enumerate(dataloader):
    item = item.to(DEVICE)

    # Generate some fake paths
    noise = torch.rand_like(item[:,0:1,:,:])
    noise = torch.cat((noise, item[:,1:,:,:]), axis=1)
    # noise = item[:,1:,:,:]
    fake = gen(noise)

    out = torch.flip(item, [1])     # map, endpoints, truth
    out = torch.cat((out, fake), axis=1)  # map, endpoints, truth, generated (raw)

    out = out.to('cpu')
    outputs = torch.cat((outputs, out), axis=0)
    if item_idx == NUM_SAMPLES:
        break

# Channels: map, endpoints, ground truth, generated (raw)
outputs = outputs[:NUM_SAMPLES,:,:,:]
maps = outputs[:NUM_SAMPLES,:1,:,:]
maps = np.squeeze(maps)
maps[maps > 0] = 0
maps[maps < 0] = 1

round outputs using Rachael's code

In [ ]:
# For each item, find start/end points using direct path (where direct path == 2)
# outputs = torch.tensor(outputs)
smooth = torch.zeros_like(outputs[:,-1:,:,:])
for i in range(outputs.shape[0]):
    # Find start/end
    ends = torch.nonzero(outputs[i,1,:,:] > 1)
    # ends = torch.flip(ends, [0])

    smooth[i] = path_to_tensor(smoothen_paths(outputs[i,-1:,:,:], ends[0], ends[1], smooth_val=SMOOTH_VAL, display=False)[0], outputs[i,-1,:,:].shape)[None,None,:,:]


outputs = torch.cat((outputs, smooth), axis=1)

# Channels: map, endpoints, ground truth, generated (raw), generated (smooth)

Format and save input, raw output, and rounded path

In [ ]:
outputs = np.asarray(outputs)

fig = plt.figure(figsize=(max(4, 4*outputs[0,0,:,:].shape[1]/FIG_SCALE),max(NUM_SAMPLES*(outputs[0,0,:,:].shape[1]/outputs[0,0,:,:].shape[0]), NUM_SAMPLES*outputs[0,0,:,:].shape[0]/FIG_SCALE)))

col_title = True
for i in range(NUM_SAMPLES):

    endpoints = np.squeeze(np.nonzero(outputs[i,1,:,:] == 2))

    # Show map
    sub = fig.add_subplot(NUM_SAMPLES,5,i*5+1)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(maps[i])
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    if col_title:
        plt.title('Obstacles')

    # show real path
    sub = fig.add_subplot(NUM_SAMPLES,4,i*4+2)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(np.stack((outputs[i,2,:,:], np.zeros_like(outputs[i,2,:,:]), maps[i]), axis=-1))
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    if col_title:
        plt.title('Truth')

    # show raw gan output
    sub = fig.add_subplot(NUM_SAMPLES,4,i*4+3)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(np.stack((outputs[i,3,:,:], np.zeros_like(outputs[i,3,:,:]), maps[i]), axis=-1))
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    if col_title:
        plt.title('Generated')

    # Show smoothened gan output
    sub = fig.add_subplot(NUM_SAMPLES,4,i*4+4)
    sub.set_xticks([])
    sub.set_yticks([])
    plt.imshow(np.stack((outputs[i,-1,:,:], np.zeros_like(outputs[i,4,:,:]), maps[i]), axis=-1))
    plt.plot(endpoints[1,:],endpoints[0,:], 'g.')
    if col_title:
        plt.title('Smoothened')

    col_title = False

plt.show()

# Check how frequently the generated paths are continuous

The following cells contain code written by Rachael

In [ ]:
def make_eq(p1, p2):
    m = (p2[1]-p1[1])/(p2[0]-p1[0])
    b = p1[1]-(m*p1[0])
    return m, b

In [ ]:
def calc_distance(p1,p2):
    return math.sqrt((p2[0]-p1[0])**2 + (p2[1]-p1[1])**2)

In [ ]:
def detailed_check(point, obstacle_coords, radius_of_obs):
    max_obstacle_dis = math.sqrt(radius_of_obs**2 + radius_of_obs**2)

    for ob_coord in obstacle_coords:
        dis = calc_distance(point, ob_coord)

        if (dis <= max_obstacle_dis):
            # calculate angle btw this point and obstacle point
            theta = math.acos(abs(point[0]-ob_coord[0])/dis)
            # calculate length of hypotenuse
            hypot = radius_of_obs/math.cos(theta)
            # if hypotenuse >= dis --> this point intersects with an obstacle
            if hypot >= dis:
                return 0
    return 1

In [ ]:
def check_btw_points(p1, p2, step, obstacle_coords, radius_of_obs, test=False):
    dis_btw_points = calc_distance(p1,p2)
    
    num_points_to_check = dis_btw_points/step
    if num_points_to_check == 0:
        num_points_to_check = 1

    if p2[0]-p1[0] != 0:            # if there is a difference in the x-axis
        
        coord_step = (p2[0]-p1[0])/num_points_to_check
        m,b = make_eq(p1, p2)

        for x in np.arange(p1[0], p2[0], coord_step):
            y = m*x + b

            if not detailed_check([x,y], obstacle_coords, radius_of_obs):
                return 0
    
    elif p2[1]-p1[1] != 0:          # if there is a difference in the y-axis (i.e. path with undefined slope)
        
        coord_step = (p2[1]-p1[1])/num_points_to_check

        for y in np.arange(p1[1], p2[1], coord_step):
            x = p1[0]

            if not detailed_check([x,y], obstacle_coords, radius_of_obs):
                return 0

    else:                           # if there is no difference in either x or y axises (i.e. the same point)
        if not detailed_check(p1, obstacle_coords, radius_of_obs):
            return 0

    return 1

In [ ]:
# a function which takes a list of a paths' x and y coords (that include the start and goal points), a map array, and a check distance value
# returns 1 if the path does NOT intersect with an obstacle, and 0 if the path does intersect with an obstacle
def check_if_avoids_obstacles(xs, ys, map, check_dis=0.5):

    # create obstacle_coords array:
    obstacle_coords = torch.fliplr(torch.nonzero(torch.tensor(map))).tolist()

    # check if any points are on an obstacle or if any paths connecting points crosses an obstacle:
    radius_of_obs = 0.5

    for idx in range(len(xs)):
        if idx == len(xs)-1:            # checking final point
            if not check_btw_points([xs[idx],ys[idx]], [xs[idx],ys[idx]], check_dis, obstacle_coords, radius_of_obs):
                # print("Path intersects obstacles.")
                return 0
        else:                           # checking all other points
            if not check_btw_points([xs[idx],ys[idx]], [xs[idx+1],ys[idx+1]], check_dis, obstacle_coords, radius_of_obs):
                # print("Path intersects obstacles.")
                return 0

    return 1 # if none of the prev conditions are met, return 1 (path does not cross an obstacle)

Back to code which I've written

In [ ]:
try:
    if continuity >= 0:
        print(f"Predicted path continuity: {continuity:.2f}%")
except:
    CALC_CONTINUITY = True

    if CALC_CONTINUITY:
        # make predictions for entire dataset
        out = torch.Tensor()
        continuity = 0
        set_size = 0
        for item_idx, item in enumerate(dataloader):
            item = item.to(DEVICE)

            # Generate some fake paths
            noise = torch.rand_like(item[:,0:1,:,:])
            noise = torch.cat((noise, item[:,1:,:,:]), axis=1)
            # noise = item[:,1:,:,:]
            fake = gen(noise)

            out = torch.flip(item, [1])     # map, endpoints, truth
            out = torch.cat((out[:,:-1,:,:], fake), axis=1)  # map, endpoints, generated (raw)

            maps = out[:,0,:,:]
            maps[maps > 0] = 0
            maps[maps < 0] = 1

            out = out.to('cpu')



            # outputs = torch.cat((outputs, out), axis=0)
            for i in range(out.shape[0]):
                # Find start/end
                ends = torch.nonzero(out[i,1,:,:] > 1)
                # ends = torch.flip(ends, [0])

                smooth = smoothen_paths(out[i,-1:,:,:], ends[0], ends[1], smooth_val=SMOOTH_VAL, display=False)[0]

                set_size += 1
                continuity += check_if_avoids_obstacles(smooth[:,1], smooth[:,0], maps[i,:,:]) # TODO: make sure xs and ys aren't backwards 

        continuity *= 100
        continuity /= set_size

        print(f"Predicted path continuity: {continuity:.2f}%")